<a href="https://colab.research.google.com/github/PavanPasidu/ScriboSense/blob/saadha/XGBOOST_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
import pandas as pd

dataset_path = '/content/drive/My Drive/Student Summarizer/dataset/'

# Read CSV file
summaries = pd.read_csv(dataset_path + 'summaries_train.csv')
prompts = pd.read_csv(dataset_path + 'prompts_train.csv')

# Display the first few rows of the dataset
print(summaries.head())
print(prompts.head())

     student_id prompt_id                                               text  \
0  000e8c3c7ddb    814d6b  The third wave was an experimentto see how peo...   
1  0020ae56ffbf    ebad26  They would rub it up with soda to make the sme...   
2  004e978e639e    3b9047  In Egypt, there were many occupations and soci...   
3  005ab0199905    3b9047  The highest class was Pharaohs these people we...   
4  0070c9e7af47    814d6b  The Third Wave developed  rapidly because the ...   

    content   wording  
0  0.205683  0.380538  
1 -0.548304  0.506755  
2  3.128928  4.231226  
3 -0.210614 -0.471415  
4  3.272894  3.219757  
  prompt_id                                    prompt_question  \
0    39c16e  Summarize at least 3 elements of an ideal trag...   
1    3b9047  In complete sentences, summarize the structure...   
2    814d6b  Summarize how the Third Wave developed over su...   
3    ebad26  Summarize the various ways the factory would u...   

                prompt_title  \
0           

In [ ]:
! pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel

_mname = 'bert-large-uncased'
tokenizer = AutoTokenizer.from_pretrained(_mname)
model = AutoModel.from_pretrained(_mname)


In [ ]:
from xgboost import XGBClassifier, XGBRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaModel
import torch
from tqdm import tqdm

tqdm.pandas()

In [ ]:
df = summaries.merge(prompts, on='prompt_id')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7165 entries, 0 to 7164
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   student_id       7165 non-null   object 
 1   prompt_id        7165 non-null   object 
 2   text             7165 non-null   object 
 3   content          7165 non-null   float64
 4   wording          7165 non-null   float64
 5   prompt_question  7165 non-null   object 
 6   prompt_title     7165 non-null   object 
 7   prompt_text      7165 non-null   object 
dtypes: float64(2), object(6)
memory usage: 503.8+ KB


In [ ]:
def preprocess(df):
    df['text'] = df['text'].str.lower()
    df['prompt_text'] = df['prompt_text'].str.lower()
    df['prompt_question'] = df['prompt_question'].str.lower()
    return df

df = preprocess(df)

In [ ]:
columns_to_drop = ['student_id', 'prompt_id', 'prompt_title']

df = df.drop(columns=columns_to_drop)


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = model.to(device)

In [ ]:
pooling_method = 'mean'

def pool(output, method):
    if method == 'cls':
        return output.last_hidden_state[:, 0, :]
    elif method == 'pooler':
        return output.pooler_output
    elif method == 'mean':
        return output.last_hidden_state.mean(dim=1)
    elif method == 'max':
        return output.last_hidden_state.max(dim=1)[0]

In [ ]:
def bert_vectorizer(texts):
    batch_size = 84

    # tokenize texts
    encoded_input = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # compute token embeddings
    with torch.no_grad():
        for i in tqdm(range(0, len(encoded_input['input_ids']), batch_size)):
            batch = {key: val[i:i+batch_size].to(device) for key, val in encoded_input.items()}
            output = model(**batch)
            if i == 0:
                embeddings = pool(output, pooling_method).cpu().detach().numpy()
            else:
                embeddings = np.concatenate((embeddings, pool(output, pooling_method).cpu().detach().numpy()), axis=0)
    return embeddings


In [ ]:
features_to_encode = ['text', 'prompt_text']

X_train = []
X_valid = []
for feature in features_to_encode:
    X_train.append(bert_vectorizer(df_train[feature].values.tolist()))
    X_valid.append(bert_vectorizer(df_test[feature].values.tolist()))

 42%|████▏     | 29/69 [5:00:09<6:50:42, 616.07s/it]

CHECK IF X[0] AND X[1] ARE COORECT , SINCE SOME COLUMNS HAVE BEEN DROPPED

In [ ]:
def extend_features(X):
    X_sub = X[0] - X[1]
    X_mul = X[0] * X[1]
    X.extend([X_sub, X_mul])
    return X

extended_features = True
if extended_features and len(features_to_encode) > 1:
    X_train = extend_features(X_train)
    X_valid = extend_features(X_valid)

In [ ]:
# Merge features
X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

# Checking feature dimensions
print(X_train.shape)
print(X_valid.shape)

In [ ]:
params = {
    'objective': 'reg:squarederror',  # Regression task
    'eval_metric': 'rmse',            # Root Mean Squared Error
    'learning_rate': 0.1,             # Typical range: [0.01, 0.3]
    'max_depth': 5,                   # Typical range: [3, 10]
    'min_child_weight': 0,            # Controls overfitting, set as 1 for small datasets
    'gamma': 0,                       # Regularization parameter, typical range: [0, 5]
    'subsample': 0.8,                 # Fraction of samples used for training, typical range: [0.5, 1]
    'colsample_bytree': 0.8,          # Fraction of features used for training, typical range: [0.5, 1]
    'n_estimators': 300,              # Number of boosting rounds
    'early_stopping_rounds': 10,      # Stop training if performance doesn't improve for N rounds
    'random_state': 42                # Seed for reproducibility
}
content_model = XGBRegressor(**params)
content_model.fit(X_train, df_train['content'], eval_set=[(X_valid, df_test['content'])])

wording_model = XGBRegressor(**params)
wording_model.fit(X_train, df_train['wording'], eval_set=[(X_valid, df_test['wording'])])

# Predict
y_pred_c = content_model.predict(X_valid)
y_pred_w = wording_model.predict(X_valid)

# Evaluate (MSE) and MCRMSE
from sklearn.metrics import mean_squared_error

def mcrmse(y_trues, y_preds):
    # calc individual rmse
    rmses = []
    for i in range(len(y_trues)):
        mse = mean_squared_error(y_trues[i], y_preds[i])
        rmse = np.sqrt(mse)
        rmses.append(rmse)

    # calc mean of rmse
    return np.mean(rmses)


mse_c = mean_squared_error(df_test['content'], y_pred_c)
mse_w = mean_squared_error(df_test['wording'], y_pred_w)
print(f'MSE content: {mse_c:.4f}')
print(f'MSE wording: {mse_w:.4f}')
mcrmse = mcrmse([df_test['content'], df_test['wording']], [y_pred_c, y_pred_w])
print(f'MCRMSE: {mcrmse:.4f}')

In [ ]:
# Predict on test set
df_test_prompts = pd.read_csv(os.path.join(DATA_DIR, 'prompts_test.csv'))
df_test_summaries = pd.read_csv(os.path.join(DATA_DIR, 'summaries_test.csv'))

df_test = df_test_summaries.merge(df_test_prompts, on='prompt_id')

# Common preprocessing steps
df_test = preprocess(df_test)

X_test = []
for feature in features_to_encode:
    X_test.append(bert_vectorizer(df_test[feature].values.tolist()))

if extended_features and len(features_to_encode) > 1:
    X_test = extend_features(X_test)

X_test = np.concatenate(X_test, axis=1)
# X_test = bert_vectorizer(df_test['text'].values.tolist())

y_pred_c = content_model.predict(X_test)
y_pred_w = wording_model.predict(X_test)

# Save predictions
df_test['content'] = y_pred_c
df_test['wording'] = y_pred_w
df_test[['student_id', 'content', 'wording']].to_csv('submission.csv', index=False)